In [1]:
!nvidia-smi

Mon Jan  1 19:27:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
%cd {HOME}
!git clone https://github.com/ashrithjacob/GroundingDINO.git
%cd {HOME}/GroundingDINO

/content
fatal: destination path 'GroundingDINO' already exists and is not an empty directory.
/content/GroundingDINO


In [4]:
!pip install -q -e .
!pip install -q roboflow

  Preparing metadata (setup.py) ... done


In [5]:
!pip install deepface
!pip install matplotlib
!pip install cv2
!pip install face-recognition
!pip install openai
!pip install chromadb

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


###CODE STARTS HERE:

In [6]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
%cd {HOME}

/content
mkdir: cannot create directory ‘/content/weights’: File exists
/content/weights
/content


In [7]:
import os
import groundingdino
import os
import supervision as sv
import torch
from torchvision.ops import box_convert
import cv2
import numpy as np
import matplotlib.pyplot as plt
import face_recognition
from deepface import DeepFace
from deepface.commons import functions, realtime, distance as dst
from groundingdino.util.inference import (
    load_model,
    load_image,
    predict,
    annotate,
)

24-01-01 19:28:51 - Directory /root/.deepface created
24-01-01 19:28:51 - Directory /root/.deepface/weights created


In [8]:
class FaceRecognition:
    def __init__(self):
        self.HOME = os.getcwd()
        self.backends = [
            "opencv",
            "ssd",
            "dlib",
            "mtcnn",
            "retinaface",
            "mediapipe",
            "yolov8",
            "yunet",
            "fastmtcnn",
        ]
        self.models = [
            "VGG-Face",
            "Facenet",
            "Facenet512",
            "OpenFace",
            "DeepFace",
            "DeepID",
            "ArcFace",
            "Dlib",
            "SFace",
        ]
        self.CONFIG_PATH = os.path.join(
            self.HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
        )

        if (self.CONFIG_PATH, "; exist:", os.path.isfile(self.CONFIG_PATH)):
            self.WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
            self.WEIGHTS_PATH = os.path.join(self.HOME, "weights", self.WEIGHTS_NAME)
            self.MODEL = load_model(self.CONFIG_PATH, self.WEIGHTS_PATH)

    def _box_face(
        self,
        image_path,
        text_prompt="human being",
        box_threshold=0.3,
        text_threshold=0.25,
    ):
        image_source, image = load_image(image_path)
        boxes, logits, phrases = predict(
            model=self.MODEL,
            image=image,
            caption=text_prompt,
            box_threshold=box_threshold,
            text_threshold=text_threshold,
        )
        return {
            "image": image_source,
            "boxes": boxes,
            "logits": logits,
            "phrases": phrases,
        }

    @classmethod
    def inflate_box(cls, boxes):
        boxes[:, 2:] *= 1.0  # scaling factor(fix box overlap issues)
        return boxes

    def display(self, image_path):
        properties = self._box_face(image_path=image_path)
        annotated_frame = annotate(
            image_source=properties["image"],
            boxes=properties["boxes"],
            logits=properties["logits"],
            phrases=properties["phrases"],
        )
        sv.plot_image(annotated_frame, (16, 16))

    def get_xyxy_mask(self, image_path):
        properties = self._box_face(image_path=image_path)
        img = properties["image"]
        boxes = properties["boxes"]
        boxes = FaceRecognition.inflate_box(boxes)
        h, w, _ = img.shape
        boxes = boxes * torch.Tensor([w, h, w, h])
        xyxy = box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy").numpy()
        xyxy = [tuple(map(int, map(round, mask))) for mask in xyxy]
        return img, xyxy

    def crop(self, image_path):
        img_list = []
        img, masks = self.get_xyxy_mask(image_path)
        for mask in masks:
            x1, y1, x2, y2 = map(int, map(round, mask))
            img_list.append(img[y1:y2, x1:x2])
        return img_list

    def deepface_box_face(self, image_path):
        boxes = []
        phrase = []
        confidence = []
        embedding_obj = DeepFace.represent(
            img_path=image_path,
            model_name=self.models[2],
            detector_backend=self.backends[4],
            enforce_detection=True,
        )
        image_source, _= load_image(image_path)
        h, w, _ = image_source.shape
        for e in embedding_obj:
            box = list(e["facial_area"].values())
            box = [x / w if i % 2 == 0 else x / h for i, x in enumerate(box)]
            box[0] += box[2]/2
            box[1] += box[3]/2
            boxes.append(box)
            confidence.append(e["face_confidence"])
            phrase.append("face_confidence")
        return {
            "image": image_source,
            "boxes": torch.Tensor(boxes),
            "logits": torch.Tensor(confidence),
            "phrases": phrase,
        }

    def display_deepface(self, image_path):
        properties = self.deepface_box_face(image_path=image_path)
        annotated_frame = annotate(
            image_source=properties["image"],
            boxes=properties["boxes"],
            logits=properties["logits"],
            phrases=properties["phrases"],
        )
        sv.plot_image(annotated_frame, (16, 16))

    def create_face_embedding(self, image_path):
        embeddings = []
        embedding_obj = DeepFace.represent(
            img_path=image_path,
            model_name=self.models[2],
            detector_backend=self.backends[4],
            enforce_detection=False,
        )
        embeddings = [e["embedding"] for e in embedding_obj]
        return embeddings


# initialize object
model = FaceRecognition()

final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [9]:
model = FaceRecognition()
img_path = "img-71.jpg"
base_name, extension = img_path.split(".")

final text_encoder_type: bert-base-uncased


In [10]:
%ls
img_list = model.crop(img_path)
crop_list = []
for idx,img in enumerate(img_list):
  new_img_path = f"{base_name}-p{idx}.{extension}"
  crop_list.append(new_img_path)
  plt.imsave(new_img_path, img)
  plt.show()

GroundingDINO/  img-71.jpg  sample_data/  weights/


In [11]:
import chromadb
from chromadb.utils import embedding_functions
from chromadb.config import Settings
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [65]:
class PersonaRecognition:
    def __init__(self, path_to_db="./"):
        openai_ef = embedding_functions.OpenAIEmbeddingFunction(
            api_key=os.getenv("OPENAI_API_KEY"), model_name="text-embedding-ada-002"
        )
        # setup Chroma in-memory, for easy prototyping. Can add persistence easily!
        self.client = chromadb.Client(Settings(persist_directory="./"))
        # Create collection. get_collection, get_or_create_collection, delete_collection also available!
        self.collection = self.client.get_or_create_collection(
            "json", embedding_function=openai_ef
        )
        self.client = chromadb.PersistentClient(path=path_to_db)


    def delete_entries(self, ids):
        self.collection.delete(ids = ids)

    def clear_db(self):
      ids = [str(idx) for idx in range(self.get_entry_numbers())]
      self.delete_entries(ids)

    def get_entry_numbers(self):
        number_of_entries = self.collection.count()
        return number_of_entries

    def add(self, feature_list):
        number_of_entries = self.get_entry_numbers()
        self.collection.add(
            documents=[
                str(feature_list)
            ],  # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
            # metadatas=[{"key":key_value}], # filter on these!
            ids=[str(number_of_entries)],  # unique for each document
        )

    def retrieve(self, query, n=2):
        results = self.collection.query(
            query_texts=query,
            n_results=n,
            # where={"metadata_field": "is_equal_to_this"}, # optional filter
            # where_document={"$contains":"search_string"}  # optional filter
        )
        return results

    def remove_none_values(self, input_dict):
        # Use dictionary comprehension to filter out key-value pairs with values equal to "none"
        filtered_dict = {key: value for key, value in input_dict.items() if value.lower() != "none"}
        return filtered_dict

    def get_embedding(self, text, model="text-embedding-ada-002"):
        text = text.replace("\n", " ")
        return client.embeddings.create(input=[text], model=model).data[0].embedding

model = PersonaRecognition()

In [13]:
# run crop_list through cogvlm and get back the response
"""
Describe the person in this image. Namely regarding:
- The person's race/ethnicity based on their physical features
"""
ethnicity = """
The person in the image appears to be of African descent, based on his skin tone and facial features.
However, it's important to note that making assumptions about someone's race or ethnicity based solely on appearance can be inaccurate and may not reflect their true identity.
"""

"""
Describe the person in this image. Namely regarding:
- The person's gender
- The person's physique
"""

description_1 = """
The person in the image appears to be male, with a medium to tall height and a medium to heavy weight.
However, these are subjective observations and may not be entirely accurate.
"""

"""
Describe the person in this image. Namely regarding:
- The person's facial features
- The person's hair colour (black, brown, blonde or red)
- The person's attire
- The person's jewelery
"""

description_2 ="""
The person in the image appears to be a male with a beard and mustache.
He has dark skin and is wearing a blue headband.
He is dressed in a dark-colored hoodie.
There is no visible jewelry on him.
"""


In [14]:
# Add openai API to integrate both parts
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

json_format ="""
{
gender[male/masculine/man or female/feminine/woman]:
ethnicity[one of the follows: african, east asian, white, middle-eastern, south asian and others]:
physique:
facial_hair:
hair colour[either black, brown, blonde or red]:
attire:
jewelery:
}
"""
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant that injests textual information and organize it as a json. render value as 'none' if no information is found"},
    {"role": "user", "content": f"organise the following data into  a json, with the following key values.\n format:{json_format}\n data:{ethnicity};{description_1};{description_2}"}
  ]
)
print(response.choices[0].message.content)
# Add time stamp to json

{
  "gender": "male",
  "ethnicity": "african",
  "physique": "medium to tall height and medium to heavy weight",
  "facial_hair": "beard and mustache",
  "hair colour": "none",
  "attire": "dark-colored hoodie with a blue headband",
  "jewelery": "none"
}


In [56]:
#Run amtching of persona
dictionary1 = {
        "hair_colour": "black",
    "gender": "male/masculine/man",
    "ethnicity": "african",
    "physique": "medium to large build",
    "facial_hair":"noticeable beard and mustache",
}

dictionary3 = {
        "hair_colour": "black",
    "gender": "male/masculine/man",
    "ethnicity": "african",
    "physique": "small build",
    "facial_hair": "no facial hair",
}

dictionary4 = {
        "hair_colour": "brown",
    "gender": "male/masculine/man",
    "ethnicity": "african",
    "height": "medium to large build",
    "facial_hair": "no facial hair",
}

In [57]:
if model.get_entry_numbers():
    model.clear_db()

In [66]:
dictionary3 = model.remove_none_values(dictionary3)
dictionary4 = model.remove_none_values(dictionary4)
# model.add_features(dictionary1)
model.add(dictionary3)
model.add(dictionary4)
print(model.get_entry_numbers())
results = model.retrieve(str(dictionary1), n=2)

6


In [67]:
THRESHOLD = 0.03
if results["distances"][0][0] < THRESHOLD:
  model.add(model.remove_none_values(dictionary1))
  print("ADDED ELEMENT")

ADDED ELEMENT


In [122]:
# TODO:
# - READ on if adding words at the end of statements affect embedding
# - Change to free embedding model
# - Try experiments
# Add scene to KG (as usual)
# Add the name to the image description before saving to KG and add timestamp
"""
Prompt for scene(as in KG pipeline):
Describe what you see here? answer regarding the following:
Where is this picture taken (environment)
 Who are the people in the picture?
 What are the activities/actions performed?
DO NOT MAKE ASSUMPTIONS ABOUT THE IMAGE, SIMPLY STATE THE FACTS IN THE IMAGE
Narrate the description in first person. If only partial body parts are visible assume that to my mine
"""
"""
Answer:
The image is taken inside a store, possibly a supermarket or a department store. There are decorations hanging from the ceiling, including heart-shaped and clover-shaped items.
In the foreground, there's a man wearing a hoodie and a blue beanie, looking directly at the camera. In the background, there's another person, possibly a woman, wearing a blue jacket.
The store appears to be well-lit with overhead lights.
"""

"\nAnswer:\nThe image is taken inside a store, possibly a supermarket or a department store. There are decorations hanging from the ceiling, including heart-shaped and clover-shaped items.\nIn the foreground, there's a man wearing a hoodie and a blue beanie, looking directly at the camera. In the background, there's another person, possibly a woman, wearing a blue jacket.\nThe store appears to be well-lit with overhead lights.\n"

In [ ]:
#Query and matching